# Temperature, scaling, enzyme kinetics, and exponential growth rate

### 1. Introduction and Historical Context

#### 1.1. Arrhenius Equation

One of the earliest attempts to capture the temperature dependence of reaction rates was pioneered by **Jacobus Henricus van ’t Hoff** (1852–1911), who observed an exponential increase in the rate of chemical reactions with increasing temperature. Later, **Svante Arrhenius** (1859–1927) formalized this relationship in the well-known **Arrhenius Equation**:

$$\displaystyle k = A \exp\Bigl( -\frac{E_a}{RT} \Bigr),$$

where  
- $k$ is the rate constant,  
- $A$ is the pre-exponential (frequency) factor,  
- $E_a$ is the activation energy,  
- $R$ is the universal gas constant, and  
- $T$ is the absolute temperature (in Kelvin).

For **enzyme-catalyzed reactions**, the same principle applies for the **catalytic rate constant** $k_{\mathrm{cat}}$, but enzymes also have a **thermal denaturation** process at higher temperatures, causing activity to drop above a certain threshold.

#### 1.2. Eyring Equation (Transition State Theory)

In the 1930s, **Henry Eyring** (1901–1981) and others developed the **Transition State Theory (TST)**. The key result of TST is the **Eyring-Polanyi Equation**, usually written in the form:

$$\displaystyle k = \kappa \frac{k_B \, T}{h} \exp\Bigl( \frac{\Delta S^\ddagger}{R}\Bigr) \exp\Bigl( - \frac{\Delta H^\ddagger}{RT} \Bigr),$$

where  
- $\kappa$ is the transmission coefficient (often assumed to be ~1 for many simple reactions),  
- $k_B$ is the Boltzmann constant,  
- $h$ is Planck’s constant,  
- $\Delta S^\ddagger$ is the entropy of activation,  
- $\Delta H^\ddagger$ is the enthalpy of activation, and  
- $R$ and $T$ are as before.

This equation is sometimes called the “**Eyring equation**” and offers a thermodynamics-based perspective. It reduces to an Arrhenius-like form when you group constants appropriately.

#### 1.3. Application to Enzyme Kinetics

For an **enzyme-catalyzed reaction**, the rate can be written as:

$$\displaystyle \text{Rate} = k_\text{cat}(T) \times [\text{E}] \times f([\text{S}]),$$

where:
- $k_\text{cat}(T)$ is the turnover number, strongly dependent on $T$,  
- $[\text{E}]$ is the enzyme concentration,  
- $f([\text{S}])$ is a function of substrate concentration (such as the Michaelis-Menten form $\tfrac{[\text{S}]}{K_M + [\text{S}] }$).

We can model $k_\text{cat}(T)$ using either the **Arrhenius Equation** or the **Eyring Equation**. However, because real enzymes **denature** at higher temperatures, we often observe a **bell-shaped** temperature activity profile.

Now, we will:
1. Demonstrate both the **Arrhenius** and **Eyring** forms for the temperature dependence of $k_\text{cat}$.  
2. Include a simple logistic denaturation function to mimic the loss of enzyme activity at high temperature.  
3. Visualize the resulting enzyme activity curves and explore parameter effects.

## 2. Constants and Helper Functions

We'll define relevant physical constants and implement two functions for the temperature-dependent rate constants:

1. **Arrhenius Rate**:  
$$\displaystyle k_{\text{Arrhenius}}(T) = A \exp\Bigl(-\frac{E_a}{RT}\Bigr).$$

2. **Eyring Rate**:  
$$\displaystyle k_{\text{Eyring}}(T) = \kappa \frac{k_B T}{h} \exp\Bigl(\frac{\Delta S^\ddagger}{R}\Bigr) \exp\Bigl(-\frac{\Delta H^\ddagger}{RT}\Bigr).$$

(Assume $\kappa \approx 1$ for simplicity.)

In both cases, we’ll multiply by a logistic **denaturation factor** to represent the fraction of enzyme that remains active at high temperature.  

Refer to **panel (c)** in the attached figure for how the Eyring equation can be fitted to data to extract thermodynamic parameters.

In [2]:
# Physical constants
R = 8.314            # J/(mol*K) - Universal gas constant
k_B = 1.380649e-23   # J/K - Boltzmann constant
h = 6.62607015e-34   # J*s - Planck's constant

def arrhenius_rate(T_celsius, A=1e7, Ea=5e4, T_denature=60.0, slope=0.5):
    """
    Returns the enzyme-catalyzed reaction rate (arbitrary units)
    using the Arrhenius form combined with a logistic denaturation factor.
    
    Parameters
    ----------
    T_celsius : float
        Temperature in Celsius
    A : float
        Pre-exponential factor for Arrhenius
    Ea : float
        Activation energy in J/mol
    T_denature : float
        Approximate denaturation temperature in Celsius
    slope : float
        Controls the steepness of the logistic denaturation
    
    Returns
    -------
    rate : float
        Modeled enzyme rate at the given temperature
    """
    T_kelvin = T_celsius + 273.15
    
    # Arrhenius part
    k_cat = A * np.exp(-Ea / (R * T_kelvin))
    
    # Logistic denaturation factor
    fraction_active = 1.0 / (1.0 + np.exp(slope * (T_celsius - T_denature)))
    
    return k_cat * fraction_active

def eyring_rate(T_celsius, dH=5e4, dS=-100, T_denature=60.0, slope=0.5):
    """
    Returns the enzyme-catalyzed reaction rate (arbitrary units)
    using the Eyring (transition state) form combined with a logistic denaturation factor.
    
    Parameters
    ----------
    T_celsius : float
        Temperature in Celsius
    dH : float
        Enthalpy of activation (J/mol)
    dS : float
        Entropy of activation (J/(mol*K))
    T_denature : float
        Approximate denaturation temperature in Celsius
    slope : float
        Controls the steepness of the logistic denaturation
    
    Returns
    -------
    rate : float
        Modeled enzyme rate at the given temperature
    """
    T_kelvin = T_celsius + 273.15
    
    # Eyring part (assuming transmission coefficient ~ 1)
    # kB*T/h in s^-1, multiplied by exponent factors
    k_cat = (k_B * T_kelvin / h) * np.exp(dS / R) * np.exp(-dH / (R * T_kelvin))
    
    # Logistic denaturation factor
    fraction_active = 1.0 / (1.0 + np.exp(slope * (T_celsius - T_denature)))
    
    return k_cat * fraction_active


## 3. Interactive Plots

Below, we create two interactive plots:
1. **Arrhenius-based** temperature dependence.  
2. **Eyring-based** temperature dependence.

In both cases, you can adjust parameters such as the activation energy/enthalpy ($E_a$ or $\Delta H^\ddagger$), the entropy of activation ($\Delta S^\ddagger$), and the denaturation temperature or slope to see how the rate curve is affected.

Refer to **panel (d)** in the figure for how Michaelis–Menten kinetics specifically depends on $k_{\mathrm{cat}}$ and $K_M$ with temperature, but here we focus on the temperature dependence of $k_{\mathrm{cat}}$ itself for simplicity.

In [3]:
def plot_arrhenius(A=1e7, Ea=5e4, T_denature=60.0, slope=0.5):
    T = np.linspace(0, 100, 200)  # Temperature in Celsius
    rates = [arrhenius_rate(t, A=A, Ea=Ea, T_denature=T_denature, slope=slope) for t in T]
    
    plt.figure(figsize=(8,5))
    plt.plot(T, rates, label='Arrhenius-based Enzyme Rate')
    plt.axvline(T_denature, color='r', linestyle='--', label='Denaturation Temp')
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Rate (arbitrary units)')
    plt.title('Arrhenius + Denaturation Model')
    plt.legend()
    plt.show()

def plot_eyring(dH=5e4, dS=-100, T_denature=60.0, slope=0.5):
    T = np.linspace(0, 100, 200)  # Temperature in Celsius
    rates = [eyring_rate(t, dH=dH, dS=dS, T_denature=T_denature, slope=slope) for t in T]
    
    plt.figure(figsize=(8,5))
    plt.plot(T, rates, label='Eyring-based Enzyme Rate')
    plt.axvline(T_denature, color='r', linestyle='--', label='Denaturation Temp')
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Rate (arbitrary units)')
    plt.title('Eyring + Denaturation Model')
    plt.legend()
    plt.show()

interact(plot_arrhenius,
         A=FloatSlider(value=1e7, min=1e5, max=1e9, step=1e5, continuous_update=False),
         Ea=FloatSlider(value=5e4, min=1e4, max=2e5, step=1e4, continuous_update=False),
         T_denature=FloatSlider(value=60, min=20, max=100, step=1.0, continuous_update=False),
         slope=FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, continuous_update=False));

interact(plot_eyring,
         dH=FloatSlider(value=5e4, min=1e4, max=2e5, step=1e4, continuous_update=False),
         dS=FloatSlider(value=-100, min=-300, max=300, step=10, continuous_update=False),
         T_denature=FloatSlider(value=60, min=20, max=100, step=1.0, continuous_update=False),
         slope=FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, continuous_update=False));

interactive(children=(FloatSlider(value=10000000.0, continuous_update=False, description='A', max=1000000000.0…

interactive(children=(FloatSlider(value=50000.0, continuous_update=False, description='dH', max=200000.0, min=…

## 4. Discussion

1. **Arrhenius vs. Eyring**  
   - The **Arrhenius Equation** is an empirical formula that captures the exponential increase of reaction rate with temperature.  
   - The **Eyring Equation** derives from **Transition State Theory**, relating rate constants to thermodynamic parameters of the transition state ($\Delta H^\ddagger$, $\Delta S^\ddagger$), as depicted in panels (b) and (c) of the figure.

2. **Role of Denaturation**  
   - Both models in this notebook include a *logistic* denaturation factor to reduce active enzyme at high temperatures. Real systems can have more intricate denaturation kinetics, but this simple approach captures the key idea: above a certain threshold, enzyme function declines rapidly.

3. **Bell-Shaped Curve**  
   - Combining an **Arrhenius- or Eyring-like increase** at moderate temperatures with **denaturation** at higher temperatures typically yields the **bell-shaped** temperature-activity curve depicted in many enzyme-kinetics references.  
   - As shown in panel (d), enzymatic rates may also be viewed in the context of substrate concentration and Michaelis–Menten parameters, but the main driver at high temperature is denaturation.

4. **Parameter Interpretation**  
   - **Arrhenius**: $E_a$ (J/mol) is the activation energy barrier.  
   - **Eyring**: $\Delta H^\ddagger$ (J/mol) is the enthalpy of activation; $\Delta S^\ddagger$ (J/(mol·K)) is the entropy of activation. A large positive $\Delta S^\ddagger$ can lead to higher rates, while a large positive $\Delta H^\ddagger$ lowers the rate.

5. **Historical Significance**  
   - **Arrhenius** (1889) built on **van ’t Hoff’s** work on temperature dependence of equilibrium constants and reaction rates.  
   - **Henry Eyring** (1935) developed Transition State Theory, leading to the **Eyring-Polanyi equation**, providing a more fundamental theoretical underpinning for temperature-dependent rate processes in chemistry and biochemistry.

## 5. Conclusion

In this notebook, we have:
- Incorporated insights from a **recent review figure** (panels (a)–(e)) showing Arrhenius, Eyring, and Michaelis–Menten perspectives.  
- Shown how **Arrhenius** and **Eyring** equations each describe temperature-dependent reaction rates.  
- Applied both to a simplified **enzyme kinetics** scenario, including a denaturation term.  
- Provided interactive sliders to explore how parameters like $E_a$, $\Delta H^\ddagger$, $\Delta S^\ddagger$, and the denaturation threshold affect the enzyme’s activity profile.

The combination of these models, especially with the logistic denaturation factor, nicely illustrates the **bell-shaped** temperature dependence typically observed in real enzymes. As the attached figure shows, such analyses can be extended to full **Michaelis–Menten** kinetics (panel d) and to distinguishing **enthalpic** vs. **entropic** components of catalysis (panel e).

Next, a conceptual **derivation** of how the **temperature dependence** of **enzyme kinetics** and basic **cellular physiology** leads to a **temperature‐dependent** rate of **cell division**, and ultimately to **exponential population growth** in microbes. The argument starts at the molecular level (enzyme‐catalyzed reactions) and progresses up to the population level (exponential growth).

---

## 1. Enzymes and the Temperature Dependence of Reaction Rates

### 1.1 Arrhenius or Eyring‐Style Dependence

At the **molecular** level, many key intracellular reactions follow temperature‐dependent kinetics that often can be approximated (over a modest temperature range) by an **Arrhenius‐type** expression:

$$
k(T)  =  A\,\exp\!\Bigl(-\frac{E_a}{R\,T}\Bigr),
$$

where  
- $k(T)$ is the reaction rate constant,  
- $E_a$ is the (apparent) activation energy,  
- $A$ is a pre‐exponential factor,  
- $R$ is the gas constant,  
- $T$ is the absolute temperature.

Alternatively, if one prefers a **transition‐state theory** perspective, the **Eyring equation** can be used:

$$
k(T)  =  \frac{k_B T}{h} \,\exp\!\Bigl(\tfrac{\Delta S^\ddagger}{R}\Bigr)\,\exp\!\Bigl(-\tfrac{\Delta H^\ddagger}{R\,T}\Bigr),
$$

where $\Delta H^\ddagger$ and $\Delta S^\ddagger$ are the activation enthalpy and entropy, respectively, and $k_B$ and $h$ are Boltzmann’s and Planck’s constants. The **key takeaway** is that **reaction rates** within the cell **increase with temperature** (until a certain thermal limit).

### 1.2 Summation of Intracellular Reaction Rates

A living cell’s **growth and division** require a **multitude of coupled enzyme reactions** (for nutrient uptake, biosynthesis, energy generation, etc.). In many simplified models, **one key rate‐limiting step** (or a small set of them) dominates the overall pace of cell growth and division. If those rate‐limiting reactions follow Arrhenius‐type kinetics, then the **cellular physiological rate** (e.g., biomass production rate, cell cycle progression rate) will inherit a similar temperature dependence:

$$
\text{Physiological rate}  \propto  k(T)  \approx  A\,\exp\!\Bigl(-\tfrac{E_a}{R\,T}\Bigr).
$$

Hence, as $T$ increases (within a tolerable range), **enzyme‐catalyzed processes speed up**, and so does the **cell’s overall metabolism** and ability to replicate.

---

## 2. From Single‐Cell Growth to Division

### 2.1 Cell Division as a (Stochastic) Process

To move from enzyme kinetics to **cell‐level** behavior, note that a **single cell** must:

1. **Replicate all essential components** (DNA, proteins, membranes, etc.).  
2. Reach a certain “completion threshold” in biomass or chromosome replication.  
3. **Divide** into two daughter cells.

Because cellular biochemical events occur stochastically, the *exact* division time of any one cell can vary. However, in an **exponential (log) growth phase** with abundant nutrients, the **average** cell‐division rate $\lambda$ (divisions per unit time) is fairly constant. This average division rate is ultimately **tied** to the cumulative rate of intracellular biochemical and biosynthetic reactions, and thus inherits their temperature dependence.

### 2.2 Average Doubling Time

If $\tau_d$ is the **average doubling time** (the mean time for one cell to become two), then

$$
\lambda  =  \frac{1}{\tau_d}
  \Longleftrightarrow  
\tau_d  =  \frac{1}{\lambda}.
$$

From the **Arrhenius perspective**, we might write

$$
\tau_d(T)  \approx  \tau_{d,0}\,\exp\!\Bigl(\tfrac{E_a}{R\,T}\Bigr),
$$

so that the cell’s doubling time **decreases** with **increasing** temperature (again, within the physiological limit). Equivalently, the **division rate** $\lambda(T) = 1/\tau_d(T)$ would **increase** with temperature:

$$
\lambda(T)  \approx  \lambda_0 \,\exp\!\Bigl(-\tfrac{E_a}{R\,T}\Bigr).
$$

---

## 3. Exponential Population Growth

### 3.1 Linking Cell Division Rate to Population Growth

Now consider a well‐mixed microbial culture (in batch or chemostat conditions, but focusing on **log phase**). If each cell divides (on average) at rate $\lambda(T)$, then the **expected** number of cells $N(t)$ follows the **exponential growth law**:

$$
\frac{dN(t)}{dt}  =  \lambda(T)\,N(t),
$$
$$
\implies\quad N(t)  =  N(0)\,\exp\!\bigl[\lambda(T)\,t\bigr].
$$

This is simply the standard Malthus‐type (or exponential) growth equation, whose solution is an exponential in time, with the **growth rate** $\lambda(T)$ now an **Arrhenius‐like function** of temperature.

### 3.2 Growth Rate vs Doubling Time

It is common in microbiology to define the **specific growth rate** $\mu(T)$ via

$$
\frac{dN}{dt}  =  \mu(T)\,N,
$$

so that $\mu(T)\equiv \lambda(T)$. Equivalently, the **doubling time** is

$$
t_d  =  \frac{\ln(2)}{\mu(T)}.
$$

Thus, you will often see microbial growth curves written as

$$
N(t)  =  N(0)\,\exp\!\bigl[\mu(T)\,t\bigr],
$$

where

$$
\mu(T)  \approx  \mu_0\,\exp\!\Bigl(-\tfrac{E_a}{R\,T}\Bigr).
$$

---

## 4. Putting It All Together

1. **Intracellular Enzyme Kinetics**  
   $ \to$ Reaction rates speed up with temperature according to Arrhenius or Eyring equations.

2. **Cell‐Level Physiology**  
   $ \to$ The **cell division cycle** (DNA replication, protein synthesis, etc.) is governed by these enzymatic rates, so the **cell division rate** $\lambda(T)$ similarly increases with temperature (up to an optimum).

3. **Population Growth**  
   $ \to$ When each cell divides at rate $\lambda(T)$, the **population** follows **exponential growth** $N(t) = N(0)\,\exp[\lambda(T)\,t]$. The **specific growth rate** $\mu(T)$ is effectively the **cell division rate** and inherits the same temperature dependence.

---

## 5. Caveats and Extensions

1. **Optimal Temperature and Denaturation**  
   - Real microbes have an **optimal growth temperature**. At **high temperatures**, enzymes **denature**, and growth drops off. Thus, beyond a certain point, Arrhenius‐type behavior breaks down.

2. **Complexity in Real Systems**  
   - Many enzymatic steps contribute to growth; the overall temperature dependence can be more complicated than a simple exponential. Empirical models like the **Ratkowsky model** ($\sqrt{\mu}$ vs. $T$) or the **Cardinal model** with minimum, optimum, and maximum temperatures, are often used.

3. **Maintenance Requirements**  
   - Below or near a certain threshold temperature, the **maintenance energy** requirement may exceed the energy‐generating reactions, effectively stopping growth.

4. **Stochastic Division**  
   - In single‐cell studies (e.g., time‐lapse microscopy), you see that individual cells have **variable** interdivision times. However, at the **population level**, the *average* behavior typically follows the exponential law.

---

## 6. Final Schematic Summary

$$
\begin{aligned}
& \text{(1) Enzyme Kinetics:} \quad
   k \sim \exp\!\Bigl(-\tfrac{E_a}{R\,T}\Bigr) 
    \longrightarrow  \text{(2) Rate-Limiting Step in Cell Growth} \\
& \quad \longrightarrow \text{Division Rate:}
     \lambda(T) \sim \exp\!\Bigl(-\tfrac{E_a}{R\,T}\Bigr) 
    \longrightarrow  \text{(3) Exponential Population Growth:} \\
& \quad
   \frac{dN}{dt}  =  \lambda(T)\,N
   \quad\Longrightarrow\quad
   N(t)  =  N(0)\,\exp[\lambda(T)\,t].
\end{aligned}
$$

Thus, **starting** from the **temperature dependence** of **intracellular enzyme rates**, we **end** with a **temperature‐dependent exponential growth** law at the **population level**.

Next, a **conceptual derivation** showing how **temperature dependence** in enzyme kinetics and **cell‐level physiology** leads to **exponential microbial growth**, but now **including** the **cell size scaling factor** from the **Metabolic Theory of Ecology (MTE)**. The argument spans:

1. **Intracellular enzyme kinetics**  
2. **Cell division and doubling**  
3. **Population‐level exponential growth**  
4. **Scaling of metabolic/physiological rates with cell size ($M$)** as formulated in MTE

---

## 1. Temperature‐Dependent Enzyme Kinetics

### 1.1 Arrhenius or Eyring‐Style Dependence

At the **molecular level**, many rate‐limiting steps (enzyme‐catalyzed reactions) show a temperature dependence that can often be approximated by an **Arrhenius‐type** expression:

$$
k(T)  =  A\,\exp\!\Bigl(-\tfrac{E_a}{R\,T}\Bigr),
$$

or, from **Transition State Theory**, by the **Eyring equation**:

$$
k(T)  =  \frac{k_B\, T}{h}\,\exp\!\Bigl(\tfrac{\Delta S^\ddagger}{R}\Bigr)\,\exp\!\Bigl(-\tfrac{\Delta H^\ddagger}{R\,T}\Bigr).
$$

In all cases, **reaction rates tend to increase with temperature** (until proteins denature at very high temperatures). A cell’s **overall physiological rate** (e.g., biomass synthesis, replication of DNA, etc.) is governed by the collective effect of many such reactions.

---

## 2. Cell Division and Doubling Rate

### 2.1 Division as a (Stochastic) Process

A single microbial cell:

1. Grows (i.e., accumulates mass/biomass),  
2. Replicates essential components (DNA, ribosomes, etc.),  
3. Then **divides** once it reaches a sufficient size or completes the cell cycle.

While **stochastic** at the single‐cell level, in **log‐phase growth**, the **average** cell‐division rate (cells per unit time) is fairly constant and is determined by the underlying biochemical (enzyme) rates. If we denote the **mean cell‐division rate** by $\lambda(T)$, it inherits a similar **Arrhenius‐like** temperature dependence:

$$
\lambda(T)  \approx  \lambda_0\,\exp\!\Bigl(-\tfrac{E_a}{R\,T}\Bigr),
$$

within a biologically relevant temperature range. The **average doubling time** is then $\tau_d = 1/\lambda(T)$.

---

## 3. Exponential Population Growth

### 3.1 Linking Division Rate to Population Size

For a well‐mixed microbial culture in **exponential (log) growth**:

$$
\frac{dN}{dt}  =  \lambda(T)\,N(t),
$$
whose solution is
$$
N(t)  =  N(0)\,\exp\!\bigl[\lambda(T)\,t\bigr].
$$

Thus, the **population** grows **exponentially** at a **temperature‐dependent** rate $\lambda(T)$. Equivalently, one can define the **specific growth rate** $\mu(T) \equiv \lambda(T)$, so

$$
N(t)  =  N(0)\,\exp\!\bigl[\mu(T)\,t\bigr], 
\quad
\mu(T)  \approx  \mu_0\,\exp\!\Bigl(-\tfrac{E_a}{R\,T}\Bigr).
$$

---

## 4. Incorporating Cell Size Scaling from Metabolic Theory of Ecology (MTE)

### 4.1 Core Premise of MTE

The **Metabolic Theory of Ecology** posits that an organism’s **mass‐specific metabolic rate** depends on:

1. **Body (or cell) mass** $M$, and  
2. **Temperature** $T$.

In a classical MTE formulation, **organismal metabolic rate** $B$ scales as:

$$
B(M, T)  \approx  B_0\,M^{\,3/4}\,\exp\!\Bigl(-\tfrac{E}{k_B\,T}\Bigr),
$$

where:  
- $M$ is the organism’s mass (for microbes, think **cell mass**),  
- $E$ is an **effective activation energy** for metabolic reactions,  
- $k_B$ here is Boltzmann’s constant (not to be confused with the same symbol in TST; context determines which constant is meant).

### 4.2 Growth Rate from Metabolic Rate

A common assumption in MTE is that the **growth rate** of an organism is roughly **proportional** to its **mass‐specific** metabolic rate:

$$
\text{growth rate}  \propto  \frac{B(M,T)}{M}.
$$

Thus,

$$
\text{growth rate}  \approx  \frac{B_0\,M^{3/4}\,\exp\!\bigl(-E/(k_B\,T)\bigr)}{M}
 = 
B_0\,M^{-1/4}\,\exp\!\bigl(-E/(k_B\,T)\bigr).
$$

### 4.3 Cell Size Scaling Factor

For **microbial cells** of mass $M$, the MTE‐inspired prediction for the **specific growth rate** $\mu(T, M)$ becomes:

$$
\mu(T, M)
 \approx 
\mu_0 \,M^{-1/4}\,\exp\!\Bigl(-\tfrac{E}{k_B\,T}\Bigr),
$$
where $\mu_0$ absorbs constants like $B_0$.  

- The **$M^{-1/4}$** factor reflects how **smaller cells** typically have a **faster** mass‐specific metabolic rate (and thus can grow/divide faster), whereas larger microbial cells would have a somewhat lower per‐mass metabolic rate.

### 4.4 Exponential Growth with MTE Scaling

Plugging $\mu(T, M)$ into the **exponential growth equation**:

$$
\frac{dN}{dt}  =  \mu(T, M)\,N
\quad\Longrightarrow\quad
N(t)
 = 
N(0)\,\exp\!\Bigl[\mu(T, M)\,t\Bigr].
$$

Hence, from MTE, the **population’s** exponential growth rate now depends explicitly on **both**:

1. **Temperature** ($\exp[-E/(k_B\,T)]$ factor),  
2. **Cell size** ($M^{-1/4}$ factor).

In simpler microbial physiology models (which ignore size variation), one might set $M\approx\text{constant}$. But if comparing cells (or species) of **different typical cell sizes**, or tracking how cell size changes under various conditions, the $M^{-1/4}$ dependence becomes relevant.

---

## 5. Putting It All Together

1. **Enzymes and Temperature**  
   - Intracellular metabolic/biochemical **rates** follow an **Arrhenius‐type** or **Eyring‐type** increase with $T$.

2. **Cell‐Level Growth and Division**  
   - The **cell division** (doubling) rate $\lambda(T)$ (or $\mu(T)$) is governed by these enzyme‐limited processes.  
   - In MTE, it also **scales** with **cell mass** as $M^{-1/4}$.

3. **Population‐Level Exponential Growth**  
   - When every cell divides at rate $\mu(T,M)$, the population satisfies $\tfrac{dN}{dt} = \mu(T,M)\,N$.  
   - Thus, we get **exponential growth** at a **temperature‐ and size‐dependent** rate.

$$
\boxed{
N(t)  =  N(0)\,
\exp\!\Bigl[\mu_0\,M^{-1/4}\,\exp\!\bigl(-\tfrac{E}{k_B\,T}\bigr)\,t\Bigr].
}
$$

---

## 6. Key Takeaways

- **Classical enzyme kinetics** (Arrhenius/Eyring) underlies the **temperature dependence** of cellular processes.  
- **Cellular physiology** dictates that **division rate** scales with that enzymatic activity.  
- **Metabolic Theory of Ecology (MTE)** adds the **cell size (mass) scaling**: metabolic rate $\sim M^{3/4}$, so **growth rate** $\sim M^{-1/4}$.  
- Combined, these yield a **temperature‐ and size‐dependent** exponential growth law:  
  $$
    \mu(T,M) \sim M^{-1/4}\,\exp\!\Bigl(-\tfrac{E}{k_B\,T}\Bigr).
  $$

Thus, from the **molecular** level (enzyme kinetics) to the **population** level (exponential growth), incorporating the **cell size** factor from MTE unifies the **thermal dependence** of growth rates with **allometric (size) scaling**—providing a more holistic understanding of **microbial growth** across different **temperatures** and **cell sizes**.

Next a derivation and discussion of how **temperature dependence** (from enzyme kinetics and metabolic theory) plus **cell size scaling** can be integrated into a **logistic growth** framework. In other words, we move from the purely **exponential** growth regime (valid at low population density) to a **density‐dependent** (logistic) regime, where **carrying capacity** plays a role.

---

## 1. Recap: Temperature and Cell Size Dependence in Exponential Growth

1. **Molecular Level (Arrhenius/Eyring)**  
   $$
   k(T)  \sim  \exp\!\bigl(-E_a/(R\,T)\bigr),
   $$
   capturing how intracellular (enzyme) reaction rates accelerate with increasing temperature (up to a limit).

2. **Cell‐Division Rate**  
   $$
   \mu(T,M)  \sim  M^{-1/4}\,\exp\!\Bigl(-\tfrac{E}{k_B\,T}\Bigr)
   $$
   (from **Metabolic Theory of Ecology**, MTE), where 
   - $M$ is the cell mass,  
   - $E$ is an effective activation energy for metabolism,  
   - the factor $M^{-1/4}$ encodes allometric scaling, so smaller cells often grow faster (per unit mass).

3. **Exponential Growth** (in unlimited resources)  
   $$
   \frac{dN}{dt}  =  \mu(T,M) N 
   \quad\Longrightarrow\quad
   N(t)  =  N(0)\,\exp\!\bigl[\mu(T,M)\,t\bigr].
   $$

However, in **real** microbial cultures, resources become depleted or waste products accumulate, preventing indefinite exponential growth. This leads to the **logistic** regime.

---

## 2. Logistic Growth: The Basic Model

### 2.1 Classic Logistic Equation

The **logistic equation** introduces a **carrying capacity** $K$, which is the maximum population (or density) sustainable by the environment:

$$
\frac{dN}{dt} 
 =  r\,N\!\Bigl(1 - \frac{N}{K}\Bigr),
$$
where  
- $r$ is the (intrinsic) per‐capita growth rate in the **absence** of density‐dependent limitation,  
- $K$ is the **carrying capacity**.

**Solution** (if $r$ and $K$ are constants) is well known:

$$
N(t) 
 =  
\frac{K}{\,1 + \bigl(\tfrac{K}{N(0)} - 1\bigr)\exp(-r\,t)\,},
$$
which saturates at $N(t)\to K$ as $t\to\infty$.

### 2.2 Temperature & Size Dependence in Logistic Growth

To generalize, we can let the **intrinsic growth rate** $r$ be **temperature**‐ and **cell size**‐dependent, i.e.,

$$
r(T,M)
 \approx 
r_0\,M^{-1/4}\,\exp\!\Bigl(-\tfrac{E}{k_B\,T}\Bigr).
$$

Thus, the logistic equation becomes:

$$
\frac{dN}{dt}
 = 
r(T,M)\,N  \Bigl(1 - \frac{N}{K}\Bigr).
$$

Depending on the complexity of the system, one might also consider $K$ to be **T‐dependent**, $M$‐dependent, or both—especially if resource supply, waste removal, or cell size changes systematically with temperature or over time.

---

## 3. How It All Ties Together

### 3.1 Initial (Low‐Density) Phase: Exponential Growth

When $N \ll K$, the term $\bigl(1 - N/K\bigr)\approx 1$, and:

$$
\frac{dN}{dt}  \approx  r(T,M)\,N.
$$

Hence, the **early** dynamics are effectively **exponential**, with the rate

$$
\mu(T,M)  \approx  r(T,M) 
 \approx 
r_0\,M^{-1/4}\,\exp\!\Bigl(-\tfrac{E}{k_B\,T}\Bigr).
$$

### 3.2 Late (High‐Density) Phase: Saturation at $K$

As $N$ approaches $K$, resource depletion or waste accumulation slows growth:

$$
\frac{dN}{dt}  \to  0
\quad\text{as}\quad
N  \to  K.
$$

Eventually, $N(t)$ saturates at $K$. No matter how **fast** the intrinsic rate $r(T,M)$ is (e.g., at high temperature for small cells), the population **cannot** surpass the environment’s **carrying capacity**.

### 3.3 Possible Temperature/Size Dependence of $K$

In many real ecological or bioprocess contexts, the **carrying capacity** $K$ might also **depend on temperature** (through effects on nutrient availability, oxygen solubility, etc.) or on **cell size** (larger cells might require more resources, reducing the total cell number sustainable). A simple extension might be:

$$
K(T,M)
 = 
K_0\,f(T)\,g(M),
$$
where $f(T)$ and $g(M)$ are functions that capture how resource utilization or environment capacity scales with temperature and cell size. Then,

$$
\frac{dN}{dt}
 = 
r(T,M)\,N
 \Bigl(1 - \frac{N}{K(T,M)}\Bigr).
$$

Although **analytical** solutions become more complicated, the qualitative behavior remains **logistic**—exponential growth at low $N$, saturating at an upper limit.

---

## 4. Illustrative Equations

Putting these ideas into a **single** differential equation:

$$
\frac{dN}{dt} =
\bigl[\underbrace{r_0\,M^{-1/4}\,\exp\!\bigl(-E/(k_B\,T)\bigr)}_{\text{Temperature & size dependent rate}}\bigr] 
N(t) 
\Bigl(1 - \frac{N(t)}{K}\Bigr).
$$

- When $N(t)\ll K$, $dN/dt \approx r_0\,M^{-1/4}\,\exp[-E/(k_B\,T)]  N$.  
- As $N(t)\to K$, $dN/dt\to 0$.  

Or the more general form if $K$ also depends on $T$ and $M$:

$$
\frac{dN}{dt}
 = 
r(T,M)\,N \Bigl(1 - \frac{N}{\,K(T,M)\!}\Bigr).
$$

---

## 5. Biological Interpretation

1. **Early Growth Phase (Exponential)**  
   - Temperature and cell‐size effects dominate, so a **small** microbe at an **optimal** temperature quickly ramps up in population.

2. **Resource Limitation (Logistic)**  
   - Eventually, the environment or culture conditions impose a **limit** $K$.  
   - Growth slows down, and the population saturates (e.g., nutrient depletion in a batch culture).

3. **Role of Cell Size**  
   - **Smaller** cells often have **higher** mass‐specific metabolic rates, thus a **steeper** initial exponential growth. However, if $K$ is in terms of total **number** of cells, smaller cells might sustain a higher $K$ (because each cell needs fewer resources). Conversely, if $K$ is in terms of **total biomass**, the absolute number of larger cells may be lower at carrying capacity.

4. **Temperature Optima**  
   - Real systems show an **optimal temperature** for growth. Extremely high $T$ leads to enzyme denaturation; extremely low $T$ slows reaction rates.  
   - Thus, the functional form $r(T,M)$ can be more complex than a simple Arrhenius. Empirical “Cardinal Temperature Models,” “Ratkowsky Model,” etc., are often used to capture the rising and falling behavior around an optimum.

---

## 6. Final Summary

- **From Molecular to Population**  
  - Enzyme kinetics and metabolic rates increase with **temperature** (Arrhenius/Eyring).  
  - **Cell‐level** growth and division rates inherit this temperature dependence and scale with **cell mass** ($M^{-1/4}$) per MTE.

- **Exponential to Logistic**  
  - At **low population densities**, growth is effectively **exponential**, with $\mu(T,M)$.  
  - At **high densities**, resources become limiting, and **logistic** growth emerges with a carrying capacity $K$.  

- **Integrated Model**  
  $$
  \frac{dN}{dt}
   = 
  \mu(T,M)\,N
  \Bigl(1 - \frac{N}{K}\Bigr)
  $$
  (optionally letting $K = K(T,M)$) succinctly unifies **temperature dependence**, **cell size scaling**, and **density limitations**.

Hence, you get a **temperature‐ and size‐dependent** logistic equation that captures both the **initial exponential ramp** (driven by temperature‐enhanced metabolism in smaller or larger cells) and the **ultimate saturation** governed by the ecological/physical constraints of the environment.

### References

1. Arrhenius, S. (1889). *Über die Reaktionsgeschwindigkeit bei Inversion von Rohrzucker durch Säuren*. **Z. Phys. Chem.** 4, 226–248.  
2. van ’t Hoff, J. H. (1884). *Études de dynamique chimique*. Frederik Muller & Cie.  
3. Eyring, H. (1935). The Activated Complex in Chemical Reactions. **J. Chem. Phys.** 3(2), 107–115.  
4. Polanyi, M. (1935). **Z. Phys. Chem. B** 28, 309–318.  
5. Johnson, K. A., & Goody, R. S. (2011). The Original Michaelis Constant: Translation of the 1913 Michaelis–Menten Paper. **Biochemistry**, 50(39), 8264–8269.  

Below is a **non‐exhaustive list** of key references and seminal works that underlie the **theoretical** and **empirical** arguments in the derivations above—covering Arrhenius and transition state theory, microbial growth models, and the Metabolic Theory of Ecology.

---

## 1. Temperature Dependence of Reaction Rates

1. **Svante Arrhenius** (1889)  
   - *Über die Reaktionsgeschwindigkeit bei Inversion von Rohrzucker durch Säuren.*  
     **Zeitschrift für Physikalische Chemie**, **4**, 226–248.  
     - Seminal paper introducing the **Arrhenius equation** for reaction rates.

2. **Henry Eyring** (1935)  
   - *The Activated Complex in Chemical Reactions.*  
     **The Journal of Chemical Physics**, **3**, 107–115.  
     - Foundational work on **Transition State Theory (TST)**, deriving the **Eyring equation**.

3. **Keith J. Laidler** (1972)  
   - *The development of the Arrhenius equation.*  
     **Journal of Chemical Education**, **39**, 333–336.  
     - Historical perspective on how the Arrhenius equation came to be.

---

## 2. Classical Microbial Growth Kinetics

1. **F. Jacob & J. Monod** (1961)  
   - *Genetic regulatory mechanisms in the synthesis of proteins.*  
     **Journal of Molecular Biology**, **3**, 318–356.  
     - While this paper is famously about gene regulation, it lays groundwork for how enzyme kinetics tie into overall cellular processes.

2. **E. O. Powell** (1956)  
   - *Growth rate and generation time of bacteria, with special reference to continuous culture.*  
     **Journal of General Microbiology**, **15**, 492–511.  
     - Early treatments of **bacterial division rates** and how they depend on environmental conditions.

3. **M. Schaechter, O. Maaløe, & N. O. Kjeldgaard** (1958 & 1962)  
   - *Dependency on medium and temperature of cell size and chemical composition during balanced growth of Salmonella typhimurium.*  
     **Journal of General Microbiology**, **19**, 592–606 (1962).  
     - Classic experimental work linking **temperature** to **cellular physiology** (cell size, composition, growth).

4. **J. D. Murray**  
   - *Mathematical Biology* (various editions).  
     - Standard reference for **logistic growth**, **exponential growth**, and **population dynamics** modeling.

---

## 3. Logistic Growth & Population Ecology

1. **Pierre F. Verhulst** (1838)  
   - *Notice sur la loi que la population poursuit dans son accroissement.*  
     **Correspondance Mathématique et Physique**, **10**, 113–121.  
     - The original paper introducing the **logistic equation**.

2. **R. Pearl & L. J. Reed** (1920)  
   - *On the rate of growth of the population of the United States since 1790 and its mathematical representation.*  
     **Proceedings of the National Academy of Sciences (USA)**, **6**, 275–288.  
     - Classic application of logistic growth to real population data.

3. **D. A. Ratkowsky, J. Olley, T. A. McMeekin, & A. Ball** (1982)  
   - *Relationship between temperature and growth rate of bacterial cultures.*  
     **Journal of Bacteriology**, **149**, 1–5.  
     - Introduced what became the **Ratkowsky model**, an empirical expression for microbial growth rates vs. temperature, often used in food microbiology.

---

## 4. Metabolic Theory of Ecology (MTE)

1. **G. B. West, J. H. Brown, & B. J. Enquist** (1997)  
   - *A general model for the origin of allometric scaling laws in biology.*  
     **Science**, **276**, 122–126.  
     - Landmark paper deriving the **quarter‐power** scaling (e.g., \(M^{3/4}\), \(M^{-1/4}\)) based on fractal‐like transport networks.

2. **G. B. West, J. H. Brown, & B. J. Enquist** (1999)  
   - *The Fourth Dimension of Life: Fractal Geometry and Allometric Scaling of Organisms.*  
     **Science**, **284**, 1677–1679.  
     - Follow‐up discussion on how scaling laws affect metabolism and other life processes.

3. **J. H. Brown, J. F. Gillooly, A. P. Allen, V. M. Savage, & G. B. West** (2004)  
   - *Toward a Metabolic Theory of Ecology.*  
     **Ecology**, **85**, 1771–1789.  
     - Synthesis article outlining how **metabolic rates** scale with **body mass** and **temperature**, forming the basis of **MTE**.

---

## 5. Integrative / Synthetic Works

1. **S. A. Levin** (Ed.)** (2009)  
   - *The Princeton Guide to Ecology.*  
     **Princeton University Press.**  
     - Contains chapters synthesizing **metabolic theory**, **population dynamics**, and **microbial ecology**.

2. **B. Kerr, M. A. Riley, M. W. Feldman, & B. J. M. Bohannan** (2002)  
   - *Local dispersal promotes biodiversity in a real‐life game of rock–paper–scissors.*  
     **Nature**, **418**, 171–174.  
     - While not specifically about temperature, it’s an example of coupling **population dynamics** with **microbial interactions**, illustrating the complexity of real ecological systems.

3. **Metzler, D. E.** (2001)  
   - *Biochemistry: The Chemical Reactions of Living Cells.* 2nd ed., **Academic Press**.  
     - Comprehensive reference on **enzyme kinetics** in the broader physiological context.

---

### Additional Notes:

- **Empirical vs. Theoretical**: Many of the earliest derivations of **temperature‐dependent growth** arose from **empirical fits** (e.g., Arrhenius, Ratkowsky) to experimental microbial data. **Transition State Theory** was largely developed in **physical chemistry** for chemical reactions but has been adapted for **biochemical** and **enzymatic** rate processes.  
- **MTE**: Extends these ideas to **whole‐organism** (or **whole‐cell**) metabolism, adding the **allometric** component (\(M^{3/4}\) scaling for metabolic rate).  
- **Logistic Growth**: Originates in **population ecology** (Verhulst, Pearl & Reed), but it’s widely applicable to **microbial cultures** in a **batch** environment with finite resources.

Together, these references provide the **theoretical foundation** and **empirical evidence** for deriving and applying **temperature‐ and size‐dependent logistic (and exponential) growth** models in microbial systems.